Dataloader ready

To Do :


Add training and validation code
add model

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split

from transformers import DistilBertTokenizerFast, DistilBertModel, DistilBertConfig

import re
import string

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn 
from transformers import AdamW
from tqdm import tqdm

import matplotlib.pyplot as plt

In [ ]:
def dataloader(train_file_path, test_file_path):
    
    print(train_file_path)
    
    train_df = pd.read_csv(train_file_path)
    test_df = pd.read_csv(test_file_path)
    
    #print(train_df.head())
    #print(test_df.head())
    
    X = train_df["excerpt"]
    Y = train_df["target"]
    
    X_test = test_df["excerpt"] 
    
    length = []
    
    for index, row in train_df.iterrows():
        length.append(len(row['excerpt']))
    
    length.sort()
    eda = pd.DataFrame(list(zip(length,Y)), columns=['length', 'target'])   
    
    print('Length of X :', len(X))
    
    token_length = np.percentile(length, 90)
    
    length = []
    
    for index, row in test_df.iterrows():
        length.append(len(row['excerpt']))
    
    length.sort()
    eda_test = pd.DataFrame(length, columns=['length'])   
    print('Length of X Test :', len(X_test))
    
    xtrain, xvalidate, ytrain, yvalidate = train_test_split(X,Y,  test_size=0.2, random_state=123)
    
    return(xtrain, ytrain, xvalidate, yvalidate, X_test) #, eda, eda_test)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

sub_file = os.path.join(dirname, filenames[0])
train_file_path = os.path.join(dirname, filenames[1])
test_file_path = os.path.join(dirname, filenames[2])
    
xtrain, ytrain, xvalidate, yvalidate, token_length, eda, eda_test = dataloader(train_file_path, test_file_path)
    
print("Token Length 90th percentile : ", token_length)

eda['length'].hist(bins=50)

eda_test['length'].hist(bins=50)

eda['target'].hist(bins=50)

In [ ]:
class text_dataset(Dataset):
    
    def __init__(self, x,y, max_length_tokens, split):
        self.x = x
        self.y = y
        self.max_length_tokens = max_length_tokens
        self.split = split
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, index):
        
        tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
        
        text = str(self.x.iloc[index])
        text = " ".join(text.split())
        
        if(self.split != 'test'):
            target = self.y.iloc[index]
        
        #print(text, target)
        
        text = ' '.join(tokenizer.tokenize(text))
        
        encoded = tokenizer.encode_plus(
            text,                    # Sentence to encode.
            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
            max_length = self.max_length_tokens,           # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',  
            truncation= True,
            )
        
       
        
        input_ids = torch.squeeze(encoded['input_ids'])
        attention_mask = torch.squeeze(encoded['attention_mask'])
        #token_type_ids= torch.squeeze(encoded['token_type_ids'])
        
        if(self.split != 'test'):
            target = torch.as_tensor(target)

            sample = {'input_ids' : input_ids, 'attention_mask' : attention_mask, 
                       'target' : target} #, 'token_type_ids': token_type_ids}
        
        else:
            sample = {'input_ids' : input_ids, 'attention_mask' : attention_mask}
            
        
        return sample
        
        
    def clean_text(self, text):
  
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text


    def text_preprocessing(self, text, tokenizer):
   
        nopunc = self.clean_text(text)
        tokenized_text = tokenizer.tokenize(nopunc)
        #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
        combined_text = ' '.join(tokenized_text)
        return combined_text

In [ ]:
class readability_model(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.config = DistilBertConfig()
       
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
               
        self.regressor = nn.Sequential(nn.LayerNorm(768, eps=1e-12),
                                       nn.Linear(768, 128),
                                       nn.ReLU(inplace=True),
                                       nn.BatchNorm1d(num_features=128),
                                       nn.Dropout(0.5),
                                       nn.Linear(128, 16),
                                       nn.ReLU(inplace=True),
                                       nn.BatchNorm1d(num_features=16),
                                       nn.Dropout(0.5),
                                       nn.Linear(16, 1))
        

        
    def forward(self, input_ids=None, attention_mask=None, head_mask=None, 
                position_ids = None, inputs_embeds=None,labels=None, output_attentions=None, 
                output_hidden_states=None, return_dict=None): #token_type_ids=None,
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        """outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            #token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = outputs[1]"""
        
        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]
        
        out = self.regressor(pooled_output)
        
        return (out)

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    
    model.cuda() #to(device)
    model.train()
    
    epoch_loss= 0.0
    
    tbar = tqdm(dataloader, unit="batch")
    
    for i, data in enumerate(tbar):
                
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        #token_type_ids = data['token_type_ids'].to(device)
        target = data['target'].to(device)

        #print(target.size())
            
        optimizer.zero_grad()
        output = model(input_ids=input_ids, attention_mask=attention_mask) #, 
                            #token_type_ids=token_type_ids)
        
        output = torch.squeeze(output)
        
        loss = criterion(output, target.float())
            
        loss.backward()
            
        optimizer.step()
            
        epoch_loss += loss.item()
            
        tbar.set_description('Train loss: %.3f' % (epoch_loss / (i+1) ))
            
    return(model, (epoch_loss/(i+1)))

In [ ]:
def validate(model, dataloader, criterion, device):
    
    model.cuda() #to(device)
    model.eval()
    
    epoch_loss= 0.0
    
    tbar = tqdm(dataloader, unit="batch")
    
    for i, data in enumerate(tbar):
                    
        input_ids = data['input_ids'].cuda() #to(device)
        attention_mask = data['attention_mask'].cuda() #to(device)
        #token_type_ids = data['token_type_ids'].to(device)
        target = data['target'].cuda() #to(device)

            #print(input_ids.size(), attention_mask.shape, token_type_ids.shape)
            
        with torch.no_grad():
            output = model(input_ids=input_ids, attention_mask=attention_mask )#, 
                            #token_type_ids=token_type_ids)
        
        output = torch.squeeze(output)
        loss = criterion(output, target.float())
            
        epoch_loss += loss.item()
            
        tbar.set_description('Validation loss: %.3f' % (epoch_loss / (i+1) ))
            
    return((epoch_loss/(i+1)))

In [ ]:
def evaluate(model, dataloader, device):
    
    model.cuda() #to(device)
    model.eval()
    
    out = []
    
    tbar = tqdm(dataloader, unit="batch")
    
    for i, data in enumerate(tbar):
                    
        input_ids = data['input_ids'].cuda() #to(device)
        attention_mask = data['attention_mask'].cuda() #to(device)
        #token_type_ids = data['token_type_ids'].to(device)
        

            #print(input_ids.size(), attention_mask.shape, token_type_ids.shape)
            
        with torch.no_grad():
            output = model(input_ids=input_ids, attention_mask=attention_mask )#, 
                            #token_type_ids=token_type_ids)
        
        output = torch.squeeze(output)
        output = output.cpu().tolist()
                    
        out.append(output)   
        
        tbar.set_description('Files: %.3f' % (i+1))
            
    return(out)

In [ ]:
def plot(y,x, title, ylabel, xlabel, savepath):

    plt.figure()

    plt.plot(x,y)
    plt.xlabel(xlabel)
    plt.title(title)
    plt.ylabel(ylabel)

    plt.savefig(savepath)

    plt.clf()

In [ ]:
def main():
    
    max_length = 512
    batch_size = 32
    epochs = 5
    
    training_loss = []
    validation_loss = []
    
    model_save_path = "readability_baseline_BERT.pth"
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

    sub_file = os.path.join(dirname, filenames[0])
    train_file_path = os.path.join(dirname, filenames[1])
    test_file_path = os.path.join(dirname, filenames[2])
    
    xtrain, ytrain, xvalidate, yvalidate, test = dataloader(train_file_path, test_file_path)
   
       
    model = readability_model()
    #model = model.to(device)
    
    
    traindata = text_dataset(x=xtrain, y=ytrain, max_length_tokens = max_length, 
                             split='train')
    validdata = text_dataset(x=xvalidate, y=yvalidate, max_length_tokens = max_length, 
                             split='validation')
    testdata = text_dataset(x=test, y=None, max_length_tokens = max_length, split='test')
    
    
   # print(traindata.__getitem__(1010))
    
    train_dataloader = DataLoader(traindata, batch_size = batch_size, shuffle= True)
    valid_dataloader = DataLoader(validdata, batch_size = batch_size, shuffle= True)
    
    test_dataloader = DataLoader(testdata, batch_size = 1, shuffle = False)
    
    optimizer = AdamW(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    
    
    for epoch in range(epochs) :
        
        print("Epoch : ", epoch)
        
        model, trainloss = train(model, train_dataloader, optimizer, criterion, device)
        print("Average Training Loss : ", trainloss)
        training_loss.append(trainloss)
        
        valid_loss = validate(model, valid_dataloader, criterion, device)
        print("Average Validation Loss : ", valid_loss)
        validation_loss.append(valid_loss)
        
        if(epoch == 0):
            torch.save(model.state_dict(), model_save_path)
            print("Initial Model Saved")
        elif(valid_loss <= min(validation_loss)):
            torch.save(model.state_dict(), model_save_path)
            print("Best Model Saved till now")
            
            
    
    
    model.load_state_dict(torch.load(model_save_path))
            
    out = evaluate(model, test_dataloader, device)
    test_ids = test_df['id']
    
    submission = pd.DataFrame(list(zip(test_ids,out)), columns=['id', 'target'])
    print(submission.head())
    
    submission.to_csv("submission.csv", index=False)
    
        

    

    
if __name__ == "__main__":
    main()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device